## Flood Event Detection (Zero-Shot Classification)
Uses a multimodal model (CLIP) that understands the relationship between text and images. This mimics a "Triage" tool where an AI filters drone images based on whether they show flooding or not.

In [1]:
import gradio as gr
from transformers import pipeline

In [2]:
classifier = pipeline(model="openai/clip-vit-base-patch32", task="zero-shot-image-classification")

def detect_flood(image):
    if image is None:
        return None

    # 2. Define our Disaster-Specific Labels
    # We don't need to retrain the model; we just tell it what to look for!
    labels = [
        "aerial view of severe flooding",
        "aerial view of dry residential area",
        "aerial view of forest or vegetation"
    ]

    # 3. Run Inference
    results = classifier(image, candidate_labels=labels)

    # 4. Format for Gradio
    # Gradio expects {label: score}
    return {res["label"]: res["score"] for res in results}

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


In [3]:
demo = gr.Interface(
    fn=detect_flood,
    inputs=gr.Image(type="pil", label="Satellite/Drone Image"),
    outputs=gr.Label(num_top_classes=3, label="Scene Analysis"),
    title="🌊 Satellite Flood Event Detection",
    description="Upload a satellite or drone image. The AI will determine if the area appears flooded, dry, or rural without any specific training."
)

In [4]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3e72dd5c8e75c5136d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Building Damage & Debris Identification (Zero-Shot Object Detection)
This uses OWL-ViT, a state-of-the-art model that can find objects (draw bounding boxes) based on text prompts. This represents a flexible "Field Deployment" tool where a responder can search for different hazards (rubble, tents, fire) dynamically.

In [5]:
from PIL import Image, ImageDraw
import numpy as np

In [6]:
detector = pipeline(model="google/owlvit-base-patch32", task="zero-shot-object-detection")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/613M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

Device set to use cpu


In [7]:
def assess_damage(image, text_queries):
    if image is None:
        return None

    # 2. Parse the text queries (e.g., "damaged building, rubble")
    # Users can enter multiple comma-separated items
    candidate_labels = [label.strip() for label in text_queries.split(",")]

    # 3. Run Inference
    predictions = detector(image, candidate_labels=candidate_labels)

    # 4. Draw Bounding Boxes
    # We use PIL to draw red boxes around detected hazards
    draw = ImageDraw.Draw(image)

    for pred in predictions:
        # Filter low confidence detections
        if pred["score"] > 0.15:
            box = pred["box"]
            label = pred["label"]
            score = pred["score"]

            # Draw box [xmin, ymin, xmax, ymax]
            draw.rectangle(
                (box["xmin"], box["ymin"], box["xmax"], box["ymax"]),
                outline="red",
                width=3
            )
            # Draw label text
            draw.text(
                (box["xmin"], box["ymin"]),
                f"{label} ({score:.2f})",
                fill="red"
            )

    return image

In [8]:
demo = gr.Interface(
    fn=assess_damage,
    inputs=[
        gr.Image(type="pil", label="Aerial Imagery"),
        gr.Textbox(
            value="damaged building, rubble, flooded street",
            label="What should the AI look for? (comma separated)"
        )
    ],
    outputs=gr.Image(label="Annotated Damage Map"),
    title="🏚️ Interactive Building Damage Identification",
    description="Type objects to detect (e.g., 'collapsed roof, debris, car'). The AI will attempt to locate them in the image."
)

In [9]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c91fe6b2af6b7b9fe9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
